In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Trade To Portfolio Rate (TTPR) Demo

This notebook shows you how to complete an inline reconciliation on 2 sets of 5 weighted instruments with different prices.

Attributes
----------
instruments
quotes
recipes
inline reconciliation
weighted instruments
"""

toggle_code("Hide docstring")

# Inline Valuation Reconciliation

This notebook shows you how to complete an inline valuation reconciliation on 2 inline portfolios of 5 weighted instruments with different prices.

- 2 sets of weighted instruments are defined with the same holding identifiers in their respective inline portfolios but with different prices.
- Inline valuations are carried out on both of these inline portfolios and reconciled against each other.

For some background, an inline portfolio is not a "real" portfolio in LUSID per se. Rather the inline portfolio holdings (instruments and weights) are defined in the inline valuation request. This allows users to run ad-hoc valuations on a set of weighted instruments without requiring a full portfolio setup.

## 1. Setup LUSID

In [2]:
# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

import os
import pandas as pd
import pytz
from datetime import datetime

pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

In [3]:
# define the apis
quotes_api = api_factory.build(lusid.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
reconciliations_api = api_factory.build(lusid.api.ReconciliationsApi)

In [4]:
# Defining variables
valuation_date = datetime(year=2022, month=3, day=8, tzinfo=pytz.UTC)
scope = "Ibor"
portfolio_code = "Inline_Recon_NB"

## 2. Load Quotes

### 2.1 Instrument Value Quotes

Load 10 prices into the quotes store with associated instrument 'ClientInternal' identifiers. These will be linked to the weighted instruments created below in Section 3.

In [5]:
# Define instrument prices
values = [20, 50, 100, 75, 60, 20, 45, 100, 85, 60]

# Define empty weighted instrument lists
weighted_instruments_a = []
weighted_instruments_b = []


for i in range(1,11):
    
    # Upsert quotes
    quotes_api.upsert_quotes(
        scope = scope,
        request_body = {f"quote_{i}": models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                models.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=f"client_internal_{i}",
                    instrument_id_type="ClientInternal",
                    quote_type="Price",
                    field="mid"
                ),
                effective_at = valuation_date
            ),
            metric_value=models.MetricValue(
                value=values[i-1],
                unit="GBP"
            )
        )}
        
    )    
    
    # Define 2 sets of 5 weighted instruments with different prices but the same holding identifiers
    if i in range(1, 6):
        weighted_instrument = lusid.WeightedInstrument(
            quantity=1,
            holding_identifier=f"client_internal_{i}",
            instrument=models.Equity(
                identifiers= lusid.EquityAllOfIdentifiers(
                    client_internal=f"client_internal_{i}",
                ),
                dom_ccy="GBP",
                instrument_type="Equity",
            ), 
        )
        weighted_instruments_a.append(weighted_instrument)

    
    if i in range(6, 11):
        weighted_instrument = lusid.WeightedInstrument(
            quantity=1,
            holding_identifier=f"client_internal_{i-5}",
            instrument=models.Equity(
                identifiers= lusid.EquityAllOfIdentifiers(
                    client_internal=f"client_internal_{i}",
                ),
                dom_ccy="GBP",
                instrument_type="Equity",
            ), 
        )
        weighted_instruments_b.append(weighted_instrument)

## 3. Inline Valuation

### 3.1 Create Valuation Recipe

Define and upsert the valuation recipe to be used by the inline valuation request.

This recipe determines that equity quotes (mid) loaded into the quotes store in scope "Ibor" for datetimes up to 24 hours before the valuation datetime are to be used for valuations.

More details on recipes can be found here: https://support.lusid.com/knowledgebase/article/KA-01895/en-us

In [6]:
# Create recipes
recipe_scope="Inline_Recon_NB"
recipe_code="Inline_Recon_NB"


# Create a recipe to perform a valuation
configuration_recipe = models.ConfigurationRecipe(
    scope=recipe_scope,
    code=recipe_code,
    market=models.MarketContext(
        market_rules=[
            # define how to resolve the quotes
            models.MarketDataKeyRule(
                key="Equity.ClientInternal.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
                quote_interval="1D.0D"
            ),
        ],
        options=models.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="Isin",
            default_scope='Lusid',
        ),
    ),
    pricing=models.PricingContext(
        options={"AllowPartiallySuccessfulEvaluation": True},
    ),
)

upsert_configuration_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=models.UpsertRecipeRequest(
        configuration_recipe=configuration_recipe
    )
)

### 3.2 Create Valuation Request 

Create inline valuation request for inline portfolios of weighted instruments for the valuation date (as above) using the recipe defined above and requesting the following 5 metrics:

1. PV
2. PV Currency
3. Holding Units
4. InstrumentId
5. Quote value

More details on valuations can be found here: https://support.lusid.com/knowledgebase/article/KA-01729/en-us

In [1]:
def generate_valuation_request(valuation_effectiveAt, instruments):

    # Create the valuation request
    valuation_request = models.InlineValuationRequest(
        recipe_id=models.ResourceId(
            scope=recipe_scope, code=recipe_code
        ),
        metrics=[
            models.AggregateSpec("Valuation/PV", "Value"),
            models.AggregateSpec("Valuation/PV/Ccy", "Value"),
            models.AggregateSpec("Holding/Units", "Value"),
            models.AggregateSpec("Analytic/default/InstrumentTag", "Value"),
            models.AggregateSpec("Quotes/Price", "Value"),
        ],
        group_by=["Analytic/default/InstrumentTag"],
        valuation_schedule=models.ValuationSchedule(
            effective_at=valuation_effectiveAt.isoformat()
        ),
        instruments=instruments
    )

    return valuation_request

### 3.3 Valuation Response

Complete inline valuation of weighted instrument sets on a given day and combine left side, right side and difference data into a single reconciliation table.

The output shows a difference in PV for 'client_internal_2' and 'client_internal_4' weighted instruments which matches up with the price difference between the left and right inline portfolios.

In [8]:
# Reconcile weighted instruments
reconciliation_respose = reconciliations_api.reconcile_inline(
    inline_valuations_reconciliation_request=models.InlineValuationsReconciliationRequest(
        left=generate_valuation_request(valuation_date, weighted_instruments_a),
        right=generate_valuation_request(valuation_date, weighted_instruments_b)))

# Format response into dataframe including left side, right side and difference data
def format_inline_recon_response(reconciliation_respose):
    recs_diff_df = pd.DataFrame(reconciliation_respose.diff)
    recs_diff_df = recs_diff_df.rename(columns={"Valuation/PV" : "PV Diff", "Analytic/default/InstrumentTag" : "ClientInternal"})
    recs_diff_df = recs_diff_df.drop(columns=['Valuation/PV/Ccy','Holding/Units', 'Quotes/Price'])
    recs_left_df = pd.DataFrame(reconciliation_respose.left.data)
    recs_left_df = recs_left_df.rename(columns={"Valuation/PV/Ccy" : "Ccy", "Valuation/PV" : "Left PV", "Holding/Units" : "Units", "Analytic/default/InstrumentTag" : "ClientInternal", "Quotes/Price" : "Left Price"})
    recs_right_df = pd.DataFrame(reconciliation_respose.right.data)
    recs_right_df = recs_right_df.rename(columns={"Valuation/PV/Ccy" : "Ccy", "Valuation/PV" : "Right PV", "Holding/Units" : "Units", "Analytic/default/InstrumentTag" : "ClientInternal", "Quotes/Price" : "Right Price"})
    recs_df = pd.merge(recs_left_df, recs_right_df, on='ClientInternal', how='left')
    recs_df = pd.merge(recs_df, recs_diff_df, on='ClientInternal', how='left')
    recs_df = recs_df.drop(columns=['Units_y','Ccy_y'])
    recs_df = recs_df.rename(columns={"Ccy_x" : "Ccy", "Units_x" : "Units"})
    recs_df = recs_df[['ClientInternal', 'Ccy', 'Units', 'Left Price', 'Left PV', 'Right Price', 'Right PV', 'PV Diff']]
    return recs_df

recon_output = format_inline_recon_response(reconciliation_respose)

# Convert df to csv
recon_output.to_csv('inline_rec.csv')

recon_output

,ClientInternal,Ccy,Units,Left Price,Left PV,Right Price,Right PV,PV Diff
0,client_internal_1,GBP,1.0,20.0,20.0,20.0,20.0,0.0
1,client_internal_2,GBP,1.0,50.0,50.0,45.0,45.0,-5.0
2,client_internal_3,GBP,1.0,100.0,100.0,100.0,100.0,0.0
3,client_internal_4,GBP,1.0,75.0,75.0,85.0,85.0,10.0
4,client_internal_5,GBP,1.0,60.0,60.0,60.0,60.0,0.0
